In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data = pd.read_csv("/Users/sunse/Documents/EST Soft Bootcamp/project-1/data/TOURISM.csv", sep=',', encoding= 'unicode_escape')

In [3]:
data.iloc[0,:]

X                                                    29227.7061
Y                                                    29549.5371
OBJECTID                                                      1
URL_PATH      www.yoursingapore.com/en/see-do-singapore/cult...
IMAGE_PATH    www.yoursingapore.com/content/dam/desktop/glob...
IMAGE_ALT_    Learn more about local Chinese culture at the ...
PHOTOCREDI                                         Joel Chua DY
PAGETITLE                  Chinatown Heritage Centre, Singapore
LASTMODIFI                        2015-11-02T10:16:52.847+08:00
LATITUDE                                                1.28351
LONGTITUDE                                            103.84435
ADDRESS                                        48 Pagoda Street
POSTALCODE                                                  NaN
OVERVIEW                          Experience how Singapore08:00
EXTERNAL_L                http://www.singaporechinatown.com.sg/
META_DESCR    At the Chinatown Heritage 

In [5]:
data["EXPECTED_TIME_SPENT"] = 60

In [6]:
new_data = data.loc[:,["OBJECTID",'LATITUDE', 'LONGTITUDE',"EXPECTED_TIME_SPENT", "PAGETITLE", "OVERVIEW", "IMAGE_PATH", "ADDRESS","OPENING_HO"]]


In [7]:
new_data

,OBJECTID,LATITUDE,LONGTITUDE,EXPECTED_TIME_SPENT,PAGETITLE,OVERVIEW,IMAGE_PATH,ADDRESS,OPENING_HO
0,1,1.283510,103.844350,60,"Chinatown Heritage Centre, Singapore",Experience how Singapore08:00,www.yoursingapore.com/content/dam/desktop/glob...,48 Pagoda Street,"Daily, 9am atown Heritage Centre, experience h..."
1,2,1.280940,103.847630,60,"Thian Hock Keng Temple, Singapore","Beautifully restored, Thian Hock Keng Temple i...",www.yoursingapore.com/content/dam/desktop/glob...,158 Telok Ayer Street,"Daily, 7.30am g Temple is the oldest Chinese t..."
2,3,1.310070,103.899420,60,Eurasian Heritage Centre: Singapore Attraction,This engaging heritage attraction takes you th...,www.yoursingapore.com/content/dam/desktop/glob...,139 Ceylon Road,"Tuesday to Sunday, 9am attraction takes you th..."
3,4,1.277219,103.837336,60,Shophouses,With many beautifully preserved examples on ha...,www.yoursingapore.com/content/dam/desktop/glob...,NaN,NaN
4,5,1.275490,103.841420,60,Pinnacle@Duxton,The soaring lines and stunning views of majest...,www.yoursingapore.com/content/dam/desktop/glob...,1 Cantonment Road,"50th storey Skybridge, Daily, 9am sky gardens..."
...,...,...,...,...,...,...,...,...,...
102,103,1.223097,103.860523,60,"Kusu Island, Singapore",If yousland,NaN,Kusu Island,NaN
103,104,1.318706,103.706442,60,"Jurong Bird Park, Singapore: Attractions & Thi...",Jurong Bird Park has almost every species to i...,www.yoursingapore.com/content/dam/desktop/glob...,2 Jurong Hill,"Daily, 8.30am - 6pm"
104,105,1.279069,103.799724,60,HortPark,Lose yourself in this dreamscape of stunning g...,www.yoursingapore.com/content/dam/desktop/glob...,33 Hyderabad Road,"Daily, 6am a gorgeous collection of stunning ..."
105,106,1.257339,103.819525,60,Adventure Cove Waterparkntent/dam/desktop/glob...,Make a big splash at this aquatic adventure pa...,www.yoursingapore.com/content/dam/desktop/glob...,"8 Sentosa Gateway, Sentosa Island","Daily, 10am ve Waterpark/language/en-US/Homepa..."


In [8]:
new_data = new_data.values

In [7]:
new_data.shape


(107, 9)

In [10]:
def bundling(data):
    # Set maximum time spent and distance for a bundle
    MAX_TIME_SPENT = 300
    MAX_DISTANCE = 10  # in km

    # Set minimum time_spent and distance for a bundle
    MIN_TIME_SPENT = 120
    MIN_DISTANCE = 5  # in km

    # Set maximum number of attraction in a bundle
    MAX_ATTRACTION = 5
    MIN_ATTRACTION = 2

    #  create a list of Attractions
    attractions = []
    for row in data:
        id = int(row[0])
        lat = float(row[1])
        lng = float(row[2])
        time_spent = int(row[3])
        page_title = str(row[4])
        overview = str(row[5])
        image_path = str(row[6])
        address = str(row[7])
        opening_hour = str(row[8])
        attractions.append((id, lat, lng, time_spent, page_title, overview, image_path, address, opening_hour))
    # Initialize the list of bundles and visited attraction
    bundles = []
    visited = set()

    # Loop through each attraction and try to bundle it with other attractions
    for i, (id, lat, lng, time_spent, page_title, overview, image_path, address, opening_hour) in enumerate(attractions):
        if id in visited:
            continue

        # Create a new bundle with the current attraction
        bundle = [(id, lat, lng, time_spent, page_title, overview, image_path, address, opening_hour)]
        visited.add(id)

        # Find all nearby attraction within 5km and add them to the bundle
        for j, (other_id, other_lat, other_lng, other_time, other_title, other_overview, other_img_path, other_address, other_open_h) in enumerate(attractions[i+1:], start=i+1):
            if other_id in visited:
                continue

            # Calculate the distance between the two attractions in km using the Haversine formula
            lat1, lng1, lat2, lng2 = np.radians(lat), np.radians(lng), np.radians(other_lat), np.radians(other_lng)
            dlat, dlng = lat2 - lat1, lng2 - lng1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng/2)**2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
            distance = 6371 * c

            if distance <= MIN_DISTANCE and len(bundle) < MAX_ATTRACTION:
                bundle.append((other_id, other_lat, other_lng, other_time, other_title, other_overview, other_img_path, other_address, other_open_h))
                visited.add(other_id)

        # Calculate the total time spent and distance of the bundle
        total_time = sum(b[3] for b in bundle)
        total_distance = sum(np.sqrt((bundle[i][1]-bundle[i-1][1])**2 + (bundle[i][2]-bundle[i-1][2])**2) for i in range(1, len(bundle)))

        # If the bundle meets the conditions, add it to the list of bundles
        if total_time >= MIN_TIME_SPENT and total_time <= MAX_TIME_SPENT and total_distance <= MAX_DISTANCE and len(bundle) >= MIN_ATTRACTION:
            bundles.append(bundle)

    return bundles


In [11]:
output = bundling(new_data)

In [65]:
print(type(output))

<class 'list'>


In [61]:
# def genRef(data):
#     result = dict()
#     for i, bundle in enumerate(data):
#         value = []
#         key = f'bundle {i+1}'
#         for id, lat, lng, time_spent, page_title, overview, image_path, address, opening_hour in bundle:
#             item = dict()
#             item["attraction number"] = id
#             item["attraction name"] = page_title
#             item["description"] = overview
#             item["image url"] = image_path
#             item["address"] = address
#             item["opening hour"] = opening_hour
#             value.append(item)
#         result[key] = value
#     return result

In [66]:
r = genRef(output)

In [67]:
r

{'bundle 1': [{'attraction number': 1,
   'attraction name': 'Chinatown Heritage Centre, Singapore',
   'description': 'Experience how Singapore08:00',
   'image url': 'www.yoursingapore.com/content/dam/desktop/global/see-do-singapore/culture-heritage/chinatown-heritage-centre-carousel01-rect.jpg',
   'address': '48 Pagoda Street',
   'opening hour': 'Daily, 9am atown Heritage Centre, experience how Singaporeinatown Heritage Centre.'},
  {'attraction number': 2,
   'attraction name': 'Thian Hock Keng Temple, Singapore',
   'description': 'Beautifully restored, Thian Hock Keng Temple is the oldest Chinese temple in Singapore and dedicated to Mazu, the Goddess of the Sea.',
   'image url': 'www.yoursingapore.com/content/dam/desktop/global/see-do-singapore/culture-heritage/thian-hock-kheng-temple-carousel01-rect.jpg',
   'address': '158 Telok Ayer Street',
   'opening hour': 'Daily, 7.30am g Temple is the oldest Chinese temple in Singapore and dedicated to Mazu, the Goddess of the Sea.'},

In [58]:
a = json.dumps(r)

In [64]:
a

'{"bundle 1": [{"attraction number": 1, "attraction name": "Chinatown Heritage Centre, Singapore", "description": "Experience how Singapore08:00", "image url": "www.yoursingapore.com/content/dam/desktop/global/see-do-singapore/culture-heritage/chinatown-heritage-centre-carousel01-rect.jpg", "address": "48 Pagoda Street", "opening hour": "Daily, 9am atown Heritage Centre, experience how Singaporeinatown Heritage Centre."}, {"attraction number": 2, "attraction name": "Thian Hock Keng Temple, Singapore", "description": "Beautifully restored, Thian Hock Keng Temple is the oldest Chinese temple in Singapore and dedicated to Mazu, the Goddess of the Sea.", "image url": "www.yoursingapore.com/content/dam/desktop/global/see-do-singapore/culture-heritage/thian-hock-kheng-temple-carousel01-rect.jpg", "address": "158 Telok Ayer Street", "opening hour": "Daily, 7.30am g Temple is the oldest Chinese temple in Singapore and dedicated to Mazu, the Goddess of the Sea."}, {"attraction number": 4, "attr

In [ ]:
# # Print the list of bundles
    # for i, bundle in enumerate(bundles):
    #     print(f"Bundle {i+1}:")
    #     for id, lat, lng, time_spent, page_title, overview, image_path, address, opening_hour in bundle:
    #         print(
    #             f"- attraction {id}: ({lat}, {lng}, {page_title},{overview},{image_path},{address},{opening_hour}), {time_spent} minutes")
